In [1]:
from utility import db_connect, query2csv
from settings import  DBNAME, DBPASS, DBUSER, DBHOST

In [2]:
qsql="""
with d as (
  select generate_series(0,6) as dayofweek
),
m as (
  select generate_series(1,12) as month
),
V_jmyl as (
  select
      baadv.analysis_area_id,
      to_char(baadv.date, 'YYYY') as year,
      avg(baadv.volume)::bigint as volume_i,
      avg(baadv.volume) as volume,
      d.dayofweek,
      m.month
  from
      baa_ex_sus.analysis_areas_daily_volume as baadv,
      d,
      m
  where     
      extract(dow from baadv.date) in (d.dayofweek)  
      AND date_part('month', baadv.date) = m.month
      group by baadv.analysis_area_id, year, d.dayofweek, m.month       
  ),
madt as (  
  select
      month,
      year,
      analysis_area_id,
      avg(volume)::bigint as volume_i,
      avg(volume) as volume
  from
      v_jmyl
      group by analysis_area_id, year, month
      having count(dayofweek)=7 -- having 7 days of data each week
),
aadt as (
select
  analysis_area_id,
  year,
  avg(volume)::bigint as AADT_i,
  round(avg(volume), 2) as AADT
from madt
  group by analysis_area_id, year
  having count(month) = 12 -- having 12 months of data
),
-- daily_exclude_holiday: daily counts for sites excluding holidays
daily_exclude_holiday as (
select
 baaad.analysis_area_id,
 baaad.date,
 baaad.volume,
 date_part('month', baaad.date) as month,
 date_part('dow', baaad.date) as dow
from
  baa_ex_sus.analysis_areas_daily_volume as baaad
  left join baa.holidays as baahd on baaad.date::date = baahd.holiday_date
where
  baahd.holiday_id is null
  group by 1,2,3
),
V_jmyl_exclude_holiday as (
  select
      baadv.analysis_area_id,
      to_char(baadv.date, 'YYYY') as year,
      avg(baadv.volume) as volume,
      d.dayofweek,
      m.month
  from
      daily_exclude_holiday as baadv,
      d,
      m
  where     
      extract(dow from baadv.date) in (d.dayofweek)  
      AND date_part('month', baadv.date) = m.month
      group by baadv.analysis_area_id, year, d.dayofweek, m.month       
),
madt_exclude_holiday as (  
  select
      month,
      year,
      analysis_area_id,
      avg(volume) as madt
  from
      V_jmyl_exclude_holiday
      group by analysis_area_id, year, month
      having count(dayofweek)=7 -- having 7 days of data each week
),
fm as (
select
  madt_nh.analysis_area_id,
  madt_nh.month,
  madt_nh.year,
  round(madt_nh.madt/aadt.aadt::numeric,2) as fm
from
  madt_exclude_holiday as madt_nh inner join aadt using(analysis_area_id, year)
where
  aadt.aadt <> 0  
),
-- fm_est: monthly factor average for the trial analysis group
fm_est as (
  select
    fg_cmt.city,
    fg_cmt.mode,
    fg_cmt.analysis_area_test_id,
    fm.year,
    fm.month,
    round(avg(fm.fm), 2) as fm_est,
    fg_cmt.random_trial_analysis_area_id_list
  from
    fm inner join baa_ex_sus.fg_city_mode_random_test as fg_cmt
    on fm.analysis_area_id = Any(fg_cmt.random_trial_analysis_area_id_list::int[])
    group by 1,2,3,4,5,7
),
wkstart as (
select
  baaad.analysis_area_id,
  date_trunc('week', baaad.date) AS week_start,
  to_char(baaad.date, 'YYYY') as year
from
  baa_ex_sus.analysis_areas_daily_volume as baaad
  left join baa.holidays as baahd on baaad.date::date = baahd.holiday_date
where
  baahd.holiday_id is null
  group by 1,2,3
  having count(baaad.date)=7
),
-- daily_no_holiday: daily counts for sites excluding a week with any holiday 
daily_no_holiday as (
select
 baaad.analysis_area_id,
 baaad.date,
 baaad.volume,
 date_part('month', baaad.date) as month,
 se.year,
 date_part('dow', baaad.date) as dow
from
  baa_ex_sus.analysis_areas_daily_volume as baaad
  inner join wkstart as se using (analysis_area_id)
where
  baaad.date <= se.week_start + interval '6' day
  and baaad.date >=se.week_start
  order by baaad.analysis_area_id, baaad.date
),
-- Vw_site: weekly volume count average for each site excluding holiday weeks
Vw_site as (
select
 analysis_area_id,
 date_trunc('week', date) as monday,
 date_part('month', date_trunc('week', date)) as month,
 year,
 round(avg(volume), 2) as Vw
from daily_no_holiday  
group by 1,2,3,4
),
-- fm_aadb_est: Estimate AADB for that week for that trial analysis area = Vw/Fm
fm_aadb_est as (
select
    fm_est.city,
    fm_est.mode,
    fm_est.analysis_area_test_id, 
    Vw_site.monday,
    Vw_site.month,
    Vw_site.year,
    Vw_site.vw,
    fm_est.fm_est,
    round(Vw_site.vw/fm_est.fm_est, 2) as fm_aadb_est,
    fm_est.random_trial_analysis_area_id_list
from
    fm_est, Vw_site
where
     fm_est.analysis_area_test_id = Vw_site.analysis_area_id
     and fm_est.year=Vw_site.year
     and fm_est.month = Vw_site.month
     and fm_est.fm_est <> 0
)
select * from fm_aadb_est
order by 3,6,5,4
"""
csvfile='5-D-fm_aadb_est_ex_sus.csv'
query2csv(qsql,csvfile)

In [3]:
qsql="""
with d as (
  select generate_series(0,6) as dayofweek
),
m as (
  select generate_series(1,12) as month
),
V_jmyl as (
  select
      baadv.analysis_area_id,
      to_char(baadv.date, 'YYYY') as year,
      avg(baadv.volume)::bigint as volume_i,
      avg(baadv.volume) as volume,
      d.dayofweek,
      m.month
  from
      baa_ex_sus.analysis_areas_daily_volume as baadv,
      d,
      m
  where     
      extract(dow from baadv.date) in (d.dayofweek)  
      AND date_part('month', baadv.date) = m.month
      group by baadv.analysis_area_id, year, d.dayofweek, m.month       
  ),
madt as (  
  select
      month,
      year,
      analysis_area_id,
      avg(volume)::bigint as volume_i,
      avg(volume) as volume
  from
      v_jmyl
      group by analysis_area_id, year, month
      having count(dayofweek)=7 -- having 7 days of data each week
),
aadt as (
select
  analysis_area_id,
  year,
  avg(volume)::bigint as AADT_i,
  round(avg(volume), 2) as AADT
from madt
  group by analysis_area_id, year
  having count(month) = 12 -- having 12 months of data
),
-- daily_exclude_holiday: daily counts for sites excluding holidays
daily_exclude_holiday as (
select
 baaad.analysis_area_id,
 baaad.date,
 baaad.volume,
 date_part('month', baaad.date) as month,
 date_part('dow', baaad.date) as dow
from
  baa_ex_sus.analysis_areas_daily_volume as baaad
  left join baa.holidays as baahd on baaad.date::date = baahd.holiday_date
where
  baahd.holiday_id is null
  group by 1,2,3
),
V_jmyl_exclude_holiday as (
  select
      baadv.analysis_area_id,
      to_char(baadv.date, 'YYYY') as year,
      avg(baadv.volume) as volume,
      d.dayofweek,
      m.month
  from
      daily_exclude_holiday as baadv,
      d,
      m
  where     
      extract(dow from baadv.date) in (d.dayofweek)  
      AND date_part('month', baadv.date) = m.month
      group by baadv.analysis_area_id, year, d.dayofweek, m.month       
),
madt_exclude_holiday as (  
  select
      month,
      year,
      analysis_area_id,
      avg(volume) as madt
  from
      V_jmyl_exclude_holiday
      group by analysis_area_id, year, month
      having count(dayofweek)=7 -- having 7 days of data each week
),
fm as (
select
  madt_nh.analysis_area_id,
  madt_nh.month,
  madt_nh.year,
  round(madt_nh.madt/aadt.aadt::numeric,2) as fm
from
  madt_exclude_holiday as madt_nh inner join aadt using(analysis_area_id, year)
where
  aadt.aadt <> 0  
),
-- fm_est: monthly factor average for the trial analysis group
fm_est as (
  select
    fg_cmt.city,
    fg_cmt.mode,
    fg_cmt.analysis_area_test_id,
    fm.year,
    fm.month,
    round(avg(fm.fm), 2) as fm_est,
    fg_cmt.random_trial_analysis_area_id_list
  from
    fm inner join baa_ex_sus.fg_city_mode_random_test as fg_cmt
    on fm.analysis_area_id = Any(fg_cmt.random_trial_analysis_area_id_list::int[])
    group by 1,2,3,4,5,7
),
wkstart as (
select
  baaad.analysis_area_id,
  date_trunc('week', baaad.date) AS week_start,
  to_char(baaad.date, 'YYYY') as year
from
  baa_ex_sus.analysis_areas_daily_volume as baaad
  left join baa.holidays as baahd on baaad.date::date = baahd.holiday_date
where
  baahd.holiday_id is null
  group by 1,2,3
  having count(baaad.date)=7
),
-- daily_no_holiday: daily counts for sites excluding a week with any holiday 
daily_no_holiday as (
select
 baaad.analysis_area_id,
 baaad.date,
 baaad.volume,
 date_part('month', baaad.date) as month,
 se.year,
 date_part('dow', baaad.date) as dow
from
  baa_ex_sus.analysis_areas_daily_volume as baaad
  inner join wkstart as se using (analysis_area_id)
where
  baaad.date <= se.week_start + interval '6' day
  and baaad.date >=se.week_start
  order by baaad.analysis_area_id, baaad.date
),
-- Vw_site: weekly volume count average for each site excluding holiday weeks
Vw_site as (
select
 analysis_area_id,
 date_trunc('week', date) as monday,
 date_part('month', date_trunc('week', date)) as month,
 year,
 round(avg(volume), 2) as Vw
from daily_no_holiday  
group by 1,2,3,4
),
-- fm_aadb_est: Estimate AADB for that week for that trial analysis area = Vw/Fm
fm_aadb_est as (
select
    fm_est.city,
    fm_est.mode,
    fm_est.analysis_area_test_id , 
    Vw_site.monday,
    Vw_site.month,
    Vw_site.year,
    Vw_site.vw,
    fm_est.fm_est,
    round(Vw_site.vw/fm_est.fm_est, 2) as fm_aadb_est,
    fm_est.random_trial_analysis_area_id_list
from
    fm_est, Vw_site
where
     fm_est.analysis_area_test_id = Vw_site.analysis_area_id
     and fm_est.year=Vw_site.year
     and fm_est.month = Vw_site.month
     and fm_est.fm_est <> 0
)
-- compute error
select
    fae.*,
    round((fae.fm_aadb_est-aadt.aadt)/aadt.aadt::numeric, 2) as error
    from
    fm_aadb_est as fae, aadt
where
    fae.analysis_area_test_id = aadt.analysis_area_id
    and fae.year = aadt.year
    and aadt.aadt <> 0
order by 1, 2, 3, 4, 6, 5
"""
csvfile='5-D-fm_aadb_error_ex_sus.csv'
query2csv(qsql,csvfile)